<a href="https://colab.research.google.com/github/parmigggiana/ml-ids/blob/main/IDS_CTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attack detection using CTF dataset

You shouldn't use 'Run All' on this notebook: in the sections 'Dimensionality Reduction' and 'Training' only run one of the subsections for each.
If you run all the cells it won't cause any error, just waste time on operations that will then be overwritten.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.cluster import KMeans, MeanShift, FeatureAgglomeration
from pandas.plotting import scatter_matrix

%matplotlib inline

## Utility Functions

In [ ]:
def print_scores(y_test, y_pred):
    auroc = metrics.roc_auc_score(y_test, y_pred)
    if auroc < 0.5:
        y_pred = [0 if x == 1 else 1 for x in y_pred] # If it's worse than random guessing we flip it - we're not cheating here, the prediction just doesn't have a concept of benign or attack, it just identifies two clusters. if auroc was less than 0.5 it means we labelled them wrong
        auroc = metrics.roc_auc_score(y_test, y_pred)
    Accuracy = metrics.accuracy_score(y_test, y_pred)
    Precision = metrics.precision_score(y_test, y_pred)
    Recall = metrics.recall_score(y_test, y_pred)
    F1 = metrics.f1_score(y_test, y_pred)
    fpr, tpr, threasholds = metrics.roc_curve(y_test, y_pred)
    """
    Confusion matrix:

        0  1 - predicted value (Wikipedia uses different convention for axes)
        0 TN FP
        1 FN TP
    """
    cf_matrix = metrics.confusion_matrix(y_test, y_pred)
    group_names = ['TN','FP','FN', 'TP']
    group_counts = [f"{value}" for value in
                cf_matrix.flatten()]
    group_percentages = [f"{value:.2f}%" for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names, group_counts, group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=labels, fmt='', cmap='mako') # thx https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea

    print(f"{Accuracy = }")
    print(f"{Precision = }")
    print(f"{Recall = }")
    print(f"Area Under ROC Curve = {auroc}")
    print(f"{F1 = }")
    plt.figure(figsize=(5, 4))
    plt.plot(fpr, tpr)
    plt.show()

In [ ]:
def load_test(scaler, dimred):
    if 'test_df' not in globals():
        test_df = pd.read_csv(filepath_or_buffer="definitive_dataset.csv")

    y_test = test_df["Label"]
    X_test = test_df.drop(columns="Label")
    X_test = scaler.transform(X_test)
    X_test = dimred.transform(X_test)

    return X_test, y_test

In [ ]:
def visualize_features(X):
    corr_matrix = pd.DataFrame(X).corr()
    plt.rcParams['figure.figsize'] = (18, 8)
    g = sns.heatmap(corr_matrix, annot=True, fmt=".1g", cmap="Greys")
    g.set_xticklabels(
        g.get_xticklabels(),
        verticalalignment="top",
        horizontalalignment="right",
        rotation=30,
    )
    plt.show()
    scatter_matrix(pd.DataFrame(X), alpha=0.2, figsize=(20, 20), diagonal="kde")

## Data preparation

In [ ]:
!wget https://github.com/parmigggiana/ml-ids/raw/main/CTF%20Data/Thu15.csv -O dataset_ctf.csv


In [ ]:
df = pd.read_csv("CTF Data/Thu15.csv")
df.shape

Make sure that there's no null rows

In [ ]:
df = df.drop(df[pd.isnull(df["Flow ID"])].index)
df.shape

Drop Label column since it's useless

In [ ]:
df.drop(columns="Label", inplace=True)
df.shape

Drop all flows pertaining ssh and caronte

In [ ]:
df.drop(df[df["Src Port"] == 22].index, inplace=True)
df.drop(df[df["Dst Port"] == 22].index, inplace=True)
df.drop(df[df["Src Port"] == 3333].index, inplace=True)
df.drop(df[df["Dst Port"] == 3333].index, inplace=True)
df.shape

Drop all flows made by our team

In [ ]:
df.drop(df[df["Src IP"].str.fullmatch(r"10\.80\.39\.\d{1,3}")].index, inplace=True)
df.drop(df[df["Dst IP"].str.fullmatch(r"10\.80\.39\.\d{1,3}")].index, inplace=True)
df.shape

In [ ]:
df["Src IP"].unique()

I noticed there's 1784 flows belonging to other addresses. This probably means there's an error in the gameserver, leaking some packets. Upon manual inspection of the pcap, I noticed they are mostly FIN/ACK and RST.
I chose to keep these flows as it's still actual traffic and we will be removing the IP features anyway

In [ ]:
df[
    ((df["Src IP"] != "10.254.0.1") & (df["Src IP"] != "10.60.39.1"))
    | ((df["Dst IP"] != "10.254.0.1") & (df["Dst IP"] != "10.60.39.1"))
].shape

The "Flow Bytes/s" and "Flow Packets/s" columns have non-numerical values, replace them.

In [ ]:
df.replace("Infinity", -1, inplace=True)
df[["Flow Bytes/s", "Flow Packets/s"]] = df[["Flow Bytes/s", "Flow Packets/s"]].apply(
    pd.to_numeric
)

Replace the NaN values and infinity values with -1.

In [ ]:
df.replace([np.inf, -np.inf, np.nan], -1, inplace=True)

7 features (Flow ID, Source IP, Source Port, Destination IP, Destination Port, Protocol, Timestamp) are excluded from the dataset. The hypothesis is that the "shape" of the data being transmitted is more important than these attributes. In addition, ports and addresses can be substituted by an attacker, so it is better that the ML algorithm does not take these features into account in training [Kostas2018].

In [ ]:
excluded = [
    "Flow ID",
    "Src IP",
    "Src Port",
    "Dst IP",
    "Dst Port",
    "Protocol",
    "Timestamp",
]
df.drop(columns=excluded, inplace=True)

## Dimensionality Reduction

We don't have labels to select features based on importance. A simple approach would be using the same features selected on the CIC-IDS-2017 Dataset

### Feature Selection

In [ ]:
features = [
    "RST Flag Count",
    "Total Length of Fwd Packet",
    "Fwd Packet Length Max",
    "Packet Length Variance",
    "Fwd Packets/s",
    "Fwd Packet Length Mean",
    "Flow IAT Max",
    "Flow Duration",
    "Flow Packets/s",
    "Total TCP Flow Time",
    "PSH Flag Count",
    "Packet Length Min",
    "Bwd IAT Total",
    "FWD Init Win Bytes",
    "Flow Bytes/s",
    "ACK Flag Count",
    "Fwd Header Length",
    "SYN Flag Count",
    "Total Bwd packets",
]
X = df[features]
scaler = RobustScaler()
scaler.fit(X)
X = scaler.transform(X)

visualize_features(X)

We can see how many of the selected features are practically independent from the dataset. This means they don't bring much information.

### PCA

In [ ]:
scaler = RobustScaler()
scaler.fit(df)
X = scaler.transform(df)

dimred = PCA(20, random_state=42)
dimred.fit(X)
X = dimred.transform(X)

visualize_features(X)

### Feature Agglomeration

In [ ]:
scaler = RobustScaler()
scaler.fit(df)
X = scaler.transform(df)

dimred = FeatureAgglomeration(n_clusters=20)
dimred.fit(X)
X = dimred.transform(X)

visualize_features(X)

## Training


In [ ]:
clf: KMeans = KMeans(2, random_state=42, verbose=10)

param_grid = {
    "algorithm": ["lloyd", "elkan"],
    "max_iter": [100, 300, 650, 1000, 2000],
    "n_init": [1, 2, 5, 10, 20],
}
search = HalvingGridSearchCV(clf, param_grid=param_grid, n_jobs=-1, verbose=10).fit(X)
clf: KMeans = search.best_estimator_

X_test, y_test = load_test(scaler, dimred)
y_pred = clf.predict(X_test)
print(search.best_estimator_)
print(search.best_params_)

### MeanShift

In [ ]:
clf = MeanShift(max_iter=300, n_jobs=-1)
clf.fit(X)

"""
MeanShift outputs 107 classes. To project them into binary I iteratively tried considering every class a benign or not, maximizing the F1 score on the test dataset (Corrected CIC-IDS-2017).
"""
X_test, y_test = load_test(scaler, dimred)
y_pred = clf.predict(X_test)

benign_classes = []
y_pred_tmp = y_pred.tolist()
classes = np.unique(y_pred)
for c in classes:
    y_pred_ben = [0 if x == c else x for x in y_pred_tmp]
    y_pred_att = [1 if x == c else x for x in y_pred_tmp]

    y_pred_ben = [1 if x != 0 else x for x in y_pred_ben]
    y_pred_att = [1 if x != 0 else x for x in y_pred_att]

    F1_B = metrics.f1_score(y_test, y_pred_ben)
    F1_A = metrics.f1_score(y_test, y_pred_att)

    print(f"F1 with class {c} as BENIGN: {F1_B}")
    print(f"F1 with class {c} as ATTACK: {F1_A}")
    print("\n")
    if F1_B > F1_A:
        benign_classes.append(c)
    y_pred_tmp = [0 if x in benign_classes else x for x in y_pred]

# print(f"{benign_classes = }")
y_pred = [1 if x != 0 else x for x in y_pred_tmp]

### Isolation Forest

In [ ]:
clf: IsolationForest = IsolationForest(
    n_estimators=200, max_features=10, bootstrap=True, random_state=42, verbose=1
)

clf.fit(X)

X_test, y_test = load_test(scaler, dimred)
y_pred = clf.predict(X_test)  # output is -1 | 1
y_pred = [0 if x == -1 else 1 for x in y_pred]

## Testing

In [ ]:
print_scores(y_test, y_pred)